In [ ]:
# Import libraries 
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob 
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os
import sys
from librosa.util import fix_length
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
CREMA = "/kaggle/input/cremad/AudioWAV/"

In [ ]:
dir_list = os.listdir(Savee)

In [ ]:
dir_list = os.listdir(CREMA)

In [ ]:
len(dir_list)

In [ ]:
emotion=[]
path=[]
for f in dir_list:
    part= f.split('.')[0]
    part=part.split('_')
    if part[2] == 'SAD':
        emotion.append('sad')
    elif part[2] == 'ANG':
        emotion.append('angry')
    elif part[2] == 'DIS':
        emotion.append('disgust')
    elif part[2] == 'FEA':
        emotion.append('fear')
    elif part[2] == 'HAP':
        emotion.append('happy')
    elif part[2] == 'NEU':
        emotion.append('neutral')
    else:
        emotion.append('Unknown')
    
    path.append(CREMA+f)
    
edf=pd.DataFrame(emotion, columns=['Emotions'])
pdf=pd.DataFrame(path, columns=['Path'])
crema_df=pd.concat([pdf,edf], axis=1)
crema_df.head()
crema_df.info()

In [ ]:
fname = crema_df.iat[150,0]
data, sample_rate = librosa.load(fname, sr=16000,duration=2.5, offset=0.6)
print(data.shape)
data = fix_length(data, size=2*sample_rate)
print(data.shape)

print(sample_rate)
plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sample_rate)

In [ ]:
ipd.Audio(fname)


In [ ]:
a=librosa.feature.melspectrogram(y=data, sr=16000)
librosa.display.specshow(a, sr=sample_rate, x_axis='linear')

In [ ]:
ps_db= librosa.power_to_db(a, ref=np.max)
librosa.display.specshow(ps_db, x_axis='s', y_axis='log')

In [ ]:
b=librosa.feature.mfcc(y=data, sr=sample_rate)
b.shape

In [ ]:
def extract_features(data,sample_rate):
    #MFCC
    mfcc = librosa.feature.mfcc(y=data, sr=16000)
    result = mfcc 
    # MelSpectogram
    mel = librosa.feature.melspectrogram(y=data, sr=16000)
    mel = librosa.power_to_db(mel, ref=np.max)
    #mel = np.mean(mel.T , axis=0)
    #plt.figure(figsize=(8, 8))
    #librosa.display.specshow(mel, x_axis='s', y_axis='log')
    result = np.vstack((result, mel)) # stacking horizontally
    #print(result.shape)
    return result

In [ ]:
max_length=32000
for path in (crema_df.Path):
    data, sample_rate = librosa.load(path,sr=16000)
    if data.shape[0] > max_length:
        max_length=data.shape[0]

In [ ]:
print(max_length)

In [ ]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

In [ ]:
def get_features(path):
    data, sample_rate = librosa.load(path,sr=16000,  duration=2.5, offset=0.6)
    data = fix_length(data, size=2*sample_rate)
    #print(data.shape)
    res1 = extract_features(data,sample_rate)
    result = np.array(res1)
    #print(result.shape)
    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data,sample_rate)
    result2 = np.array(res2) #np.vstack((result, res2)) # stacking vertically
    #print(result.shape)
    # data with stretching and pitching
    #new_data = stretch(data)
    #print(new_data.shape)
    #data_stretch_pitch = pitch(new_data, sample_rate)
    #res3 = extract_features(data_stretch_pitch,sample_rate)
    #result = np.vstack((result, res3)) # stacking vertically
      
    #print(result.shape)
    #print(result2.shape)

    return result,result2

In [ ]:
a, b = [], []
for path, emotion in zip(crema_df.Path, crema_df.Emotions):
    feature,feature2 = get_features(path)
    a.append(feature)
    a.append(feature2) #if augmentation true
    b.append(emotion)
    b.append(emotion) #if augmentation true

In [ ]:
a, b = [], []
for path, emotion in zip(crema_df.Path, crema_df.Emotions):
    feature = get_features(path)
    for i in feature:
        a.append(i)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        b.append(emotion)

In [ ]:
len(a)

In [ ]:
Features = pd.DataFrame(a)
Features['labels'] = b

In [ ]:
Features.to_csv('features_mel_only_mean_dB_augmentsiz.csv', index=False)
Features.head()

In [ ]:
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

In [ ]:
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.2, shuffle=True)
x_train, x_valid, y_train, y_valid =  train_test_split(x_train, y_train, test_size=0.2, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape

In [ ]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape

In [ ]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_valid = np.expand_dims(x_valid, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape

In [ ]:
X=np.array(a)
Y=np.array(b)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
Y[2]

In [ ]:
encoder = OneHotEncoder()
Y2 = encoder.fit_transform(Y.reshape(-1,1)).toarray()

In [ ]:
Y2[3]

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [ ]:
np.save('Features_mel_mfcc.npy', X)
create_download_link(filename='Features_mel_mfcc.npy')

In [ ]:
np.save('Labels_mel_mfcc.npy', Y2)
create_download_link(filename='Labels_mel_mfcc.npy')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y2, random_state=0, test_size=0.2, shuffle=True)
x_train, x_valid, y_train, y_valid =  train_test_split(x_train, y_train, test_size=0.2, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape

In [ ]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],-1)
x_valid=x_valid.reshape(x_valid.shape[0],x_valid.shape[1],x_valid.shape[2],-1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],-1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_valid.shape, y_valid.shape

In [ ]:
np.save('x_train.npy', x_train)
create_download_link(filename='x_train.npy')

In [ ]:
np.save('y_train.npy', y_train)
create_download_link(filename='y_train.npy')

In [ ]:
np.save('x_valid.npy', x_valid)
create_download_link(filename='x_valid.npy')

In [ ]:
np.save('y_valid.npy', y_valid)
create_download_link(filename='y_valid.npy')